- 웹 API
- 네이버 뉴스 크롤링(https://developers.naver.com/docs/serviceapi/search/news/news.md#%EB%89%B4%EC%8A%A4-%EA%B2%80%EC%83%89-%EA%B2%B0%EA%B3%BC-%EC%A1%B0%ED%9A%8C)

In [1]:
import os
import sys
import urllib.request
import datetime
import time
import json

In [2]:
# 네이버 개발자 센터에서 받은 값
client_id = 'YwMwm1qX948VxA0NB87d'
client_secret = 'UCM0_qCeUi'

In [3]:
# 받아온 url로 데이터 가져오기
def getRequestUrl(url):    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    print("request ok...")
    try: 
        print("trying ok...")
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL... : %s" % (datetime.datetime.now(), url))
        return None

In [4]:
# url 생성하기
def getNaverSearch(node, srcText, start, display):    
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    
    url = base + node + parameters    
    responseDecode = getRequestUrl(url)   # url로 요청
    
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [5]:
# 하나의 json 객체 만들기...
def getPostData(post, jsonResult, cnt):    
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    
    pDate = datetime.datetime.strptime(post['pubDate'],  '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 
'org_link':org_link,   'link': org_link,   'pDate':pDate})
    return  

In [8]:
# 크롤링하기
def main():
    node = 'news'   # 크롤링 할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  # url 생성하고 데이터 받아오기, 1번째 데이터 가져오기
    total = jsonResponse['total']
 
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):         
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt)  # 객체화하기     
        
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)  # 그 다음(start)번째 데이터 가져오기
       
    print('전체 검색 : %d 건' %total)
    
    # 저장하기
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult,  indent=4, sort_keys=True,  ensure_ascii=False)
                        
        outfile.write(jsonFile)
        
    print("가져온 데이터 : %d 건" %(cnt)) # 1000개씩 가져와지는데, 이는 네이버에서 서비스 신청할 때 정해진 것
    print ('%s_naver_%s.json SAVED' % (srcText, node))

In [9]:
if __name__ == '__main__':
    main()

검색어를 입력하세요: gpt
request ok...
trying ok...
[2023-04-12 09:56:23.054412] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:23.250920] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:23.429988] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:23.619544] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:23.804974] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:23.993769] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:24.181036] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:24.378690] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:24.574613] Url Request Success
request ok...
trying ok...
[2023-04-12 09:56:24.769614] Url Request Success
request ok...
trying ok...
HTTP Error 400: Bad Request
[2023-04-12 09:56:24.880165] Error for URL... : https://openapi.naver.com/v1/search/news.json?query=gpt&start=1001&display=100
전체 검색 : 32594 건
가져온 데이터 : 1000 건
gpt_naver